# Long Short-term Memory

In [1]:
from d2l.tensorflow import data, losses, optimizers, models

import tensorflow as tf
import numpy as np

from d2l.tensorflow import config

config.setup('CPU')

In [2]:
data_iter, vocab = data.load_seq_data('../data/timemachine.txt', 128, 1)

In [3]:
vocab_size = len(vocab)
corpus_indices = data_iter.corpus
idx_to_char, char_to_idx = vocab.idx_to_token, vocab.token_to_idx
num_inputs, num_hiddens, num_outputs = vocab_size, 512, vocab_size

def get_params():
    def _one(shape):
        return tf.random.normal(stddev=0.01, shape=shape)

    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                tf.zeros(num_hiddens))

    W_xi, W_hi, b_i = _three()  # Input gate parameters
    W_xf, W_hf, b_f = _three()  # Forget gate parameters
    W_xo, W_ho, b_o = _three()  # Output gate parameters
    W_xc, W_hc, b_c = _three()  # Candidate cell parameters
    # Output layer parameters
    W_hq = _one((num_hiddens, num_outputs))
    b_q = tf.zeros(num_outputs)
    # Create gradient
    params = [
        W_xi, W_hi, b_i, W_xf, W_hf, b_f, 
        W_xo, W_ho, b_o, W_xc, W_hc,
        b_c, W_hq, b_q
    ]
    return [tf.Variable(param) for param in params]

### State initializer

In [4]:
def init_lstm_state(batch_size, num_hiddens):
    state = [
        tf.zeros(shape=(batch_size, num_hiddens)),
        tf.zeros(shape=(batch_size, num_hiddens))
    ]
    return [tf.Variable(st) for st in state]

### LSTM Cell

In [5]:
def lstm(inputs, state, params):
    [
        W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, 
        W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q
    ] = params
    (H, C) = state
    outputs = []
    for X in inputs:
        I = tf.sigmoid(tf.matmul(X, W_xi) + tf.matmul(H, W_hi) + b_i)
        F = tf.sigmoid(tf.matmul(X, W_xf) + tf.matmul(H, W_hf) + b_f)
        O = tf.sigmoid(tf.matmul(X, W_xo) + tf.matmul(H, W_ho) + b_o)
        C_tilda = tf.tanh(tf.matmul(X, W_xc) + tf.matmul(H, W_hc) + b_c)
        C = F * C + I * C_tilda
        H = O * tf.tanh(C)
        Y = tf.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H, C)

### Train the Model

In [6]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['traveller', 'time traveller']
loss = losses.softmax_cross_entropy
optimizer = optimizers.sgd

In [7]:
# models.train_and_predict_rnn(
#     lstm, loss, optimizer, get_params, init_lstm_state, num_hiddens,
#     vocab_size, corpus_indices, idx_to_char,
#     char_to_idx, False, num_epochs, num_steps, lr,
#     clipping_theta, batch_size, pred_period, pred_len,
#     prefixes
# )